In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
import optuna

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Baseline
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from torch.utils.data import DataLoader

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [55]:
df = pd.read_csv("/Users/yehana2002/Projects/DSGP/datasets/processed/final_merged_dataset_ready.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)
df["series_id"] = "sri_lanka"
df.rename(columns={"Petrol_Price": "petrol_price"}, inplace=True)
df["time_idx"] = np.arange(len(df))

df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)

In [56]:
max_encoder_length = 36
max_prediction_length = 12
training_cutoff = df["time_idx"].max() - max_prediction_length

time_varying_known_reals = [
    "time_idx", "Date"
] + [col for col in df.columns if col not in ["petrol_price", "diesel_price", "series_id", "time_idx"] and df[col].dtype in [np.float64, np.int64]]

training = TimeSeriesDataSet(
    df[df.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="petrol_price",
    group_ids=["series_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["series_id"],
    time_varying_known_reals=time_varying_known_reals,
    time_varying_unknown_reals=["petrol_price"],
    target_normalizer=GroupNormalizer(groups=["series_id"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)

In [57]:
validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)

#### Optuna testing for Petrol model

In [58]:
def objective(trial):
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    hidden_size = trial.suggest_int("hidden_size", 8, 64)
    attention_head_size = trial.suggest_int("attention_head_size", 1, 4)
    dropout = trial.suggest_float("dropout", 0.0, 0.3)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)

    train_loader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_loader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    model = TemporalFusionTransformer.from_dataset(
        training,
        hidden_size=hidden_size,
        attention_head_size=attention_head_size,
        dropout=dropout,
        learning_rate=learning_rate,
        loss=SMAPE(),
        log_interval=0,
        reduce_on_plateau_patience=4,
    )

    trainer = Trainer(
        max_epochs=20,
        accelerator="cpu",
        logger=False,
        enable_checkpointing=False,
        callbacks=[EarlyStopping(monitor="val_loss", patience=5)],
    )

    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

    val_actuals = torch.cat([y[0] for x, y in iter(val_loader)])
    val_predictions = model.predict(val_loader)

    smape_score = SMAPE()(val_predictions, val_actuals).item()
    return smape_score

In [59]:
study = optuna.create_study(direction="minimize", study_name="TFT_Petrol_Tuning")
study.optimize(objective, n_trials=30)

print("Best SMAPE:", study.best_value)
print("Best Params:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

[I 2025-04-05 13:30:49,447] A new study created in memory with name: TFT_Petrol_Tuning
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS availabl

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 9: 100%|██████████| 2/2 [00:00<00:00,  5.78it/s, loss=0.54, train_loss_step=0.540, val_loss=0.993, train_loss_epoch=0.540] 

...

[I 2025-04-05 13:36:03,209] Trial 28 finished with value: 0.1191493347287178 and parameters: {'batch_size': 8, 'hidden_size': 15, 'attention_head_size': 2, 'dropout': 0.04387737307713756, 'learning_rate': 0.006517627670110531}. Best is trial 27 with value: 0.056701671332120895.
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU

/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 5: 100%|██████████| 7/7 [00:00<00:00, 12.89it/s, loss=0.666, train_loss_step=0.577, val_loss=0.436, train_loss_epoch=0.661]

[I 2025-04-05 13:36:06,586] Trial 29 finished with value: 0.4356025755405426 and parameters: {'batch_size': 8, 'hidden_size': 8, 'attention_head_size': 3, 'dropout': 0.09243320597303416, 'learning_rate': 0.0011426498419219344}. Best is trial 27 with value: 0.056701671332120895.



Best SMAPE: 0.056701671332120895
Best Params:
  batch_size: 8
  hidden_size: 16
  attention_head_size: 1
  dropout: 0.10501453459070806
  learning_rate: 0.007120219150765482


#### Retraining with the best parameters 

In [86]:
train_loader = training.to_dataloader(train=True, batch_size=8, num_workers=0)
val_loader = validation.to_dataloader(train=False, batch_size=8, num_workers=0)

# Define TFT with Optuna-optimized parameters
tft = TemporalFusionTransformer.from_dataset(
    training,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.10501453459070806,
    learning_rate=0.007120219150765482,
    loss=SMAPE(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

# Lightning Trainer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning import Trainer

early_stop = EarlyStopping(monitor="val_loss", patience=10, verbose=True, mode="min")
lr_logger = LearningRateMonitor()

trainer = Trainer(
    max_epochs=50,
    accelerator="cpu",
    callbacks=[early_stop, lr_logger],
    gradient_clip_val=0.1,
)

# Train the model
trainer.fit(tft, train_dataloaders=train_loader, val_dataloaders=val_loader)

/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devic

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████| 7/7 [00:01<00:00,  4.90it/s, loss=0.443, v_num=31, train_loss_step=0.302, val_loss=1.340]

...

Epoch 17: 100%|██████████| 7/7 [00:01<00:00,  4.80it/s, loss=0.0991, v_num=31, train_loss_step=0.0813, val_loss=0.0742, train_loss_epoch=0.095]

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.074


Epoch 27: 100%|██████████| 7/7 [00:01<00:00,  4.85it/s, loss=0.0635, v_num=31, train_loss_step=0.0584, val_loss=0.0857, train_loss_epoch=0.0636]

Monitored metric val_loss did not improve in the last 10 records. Best score: 0.074. Signaling Trainer to stop.


Epoch 27: 100%|██████████| 7/7 [00:02<00:00,  3.40it/s, loss=0.0635, v_num=31, train_loss_step=0.0584, val_loss=0.0857, train_loss_epoch=0.0613]


In [87]:
best_model = TemporalFusionTransformer.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
actuals = torch.cat([y[0] for x, y in iter(val_loader)])
predictions = best_model.predict(val_loader)

y_true = actuals.detach().cpu().numpy()
y_pred = predictions.detach().cpu().numpy()

print(f"\n🔍 Final Evaluation Metrics (Optuna-tuned TFT):")
print(f"SMAPE: {SMAPE()(predictions, actuals).item():.2f}")
print(f"MAE: {mean_absolute_error(y_true, y_pred):.2f}")
print(f"RMSE: {mean_squared_error(y_true, y_pred, squared=False):.2f}")
print(f"R² Score: {r2_score(y_true, y_pred):.4f}")


🔍 Final Evaluation Metrics (Optuna-tuned TFT):
SMAPE: 0.09
MAE: 34.39
RMSE: 34.39
R² Score: nan


/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/sklearn/metrics/_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


### Training TFT for petrol price prediction

In [103]:
df = pd.read_csv("/Users/yehana2002/Projects/DSGP/datasets/processed/final_merged_dataset_ready.csv")

In [104]:
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)
df["series_id"] = "sri_lanka"
df.rename(columns={"Petrol_Price": "petrol_price"}, inplace=True)
df["time_idx"] = np.arange(len(df))

df.fillna(method="ffill", inplace=True)
df.fillna(method="bfill", inplace=True)

#### Feature engineering

In [105]:
df["crude_roc"] = df["Crude_Oil_Price"].pct_change().fillna(0)
df["usd_lkr_roc"] = df["USD_LKR"].pct_change().fillna(0)

# 3-month lag
df["petrol_lag3"] = df["petrol_price"].shift(3).fillna(method="bfill")
df["crude_lag3"] = df["Crude_Oil_Price"].shift(3).fillna(method="bfill")

# Interaction terms
df["crude_usd_interaction"] = df["Crude_Oil_Price"] * df["USD_LKR"]
df["inflation_x_policy"] = df["annual_cpi_inflation_rate"] * df["policy_rate"]

#### TFT configs

In [106]:
max_encoder_length = 36
max_prediction_length = 12
training_cutoff = df["time_idx"].max() - max_prediction_length

time_varying_known_reals = [
    "time_idx", "Date"
] + [col for col in df.columns if col not in [
    "petrol_price", "diesel_price", "series_id", "time_idx"
] and df[col].dtype in [np.float64, np.int64]]

#### Time series dataset

In [107]:
training_cutoff = df["time_idx"].max() - max_prediction_length

tft_training = TimeSeriesDataSet(
    df[df.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="petrol_price",
    group_ids=["series_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["series_id"],
    time_varying_known_reals=time_varying_known_reals,
    time_varying_unknown_reals=["petrol_price"],
    target_normalizer=GroupNormalizer(groups=["series_id"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

tft_validation = TimeSeriesDataSet.from_dataset(tft_training, df, predict=True, stop_randomization=True)

#### Data Loaders

In [108]:
batch_size = 8
train_dataloader = tft_training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = tft_validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

#### Model

In [109]:
tft = TemporalFusionTransformer.from_dataset(
    tft_training,
    learning_rate=0.007120219150765482,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.10501453459070806,
    loss=SMAPE(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


#### Trainer

In [110]:
early_stop_callback = EarlyStopping(monitor="val_loss", patience=10, verbose=True, mode="min")
lr_logger = LearningRateMonitor()

trainer = Trainer(
    max_epochs=100,
    accelerator="cpu",
    callbacks=[early_stop_callback, lr_logger],
    gradient_clip_val=0.1,
)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [111]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)


   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | SMAPE                           | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 1     
3  | prescalers                         | ModuleDict                      | 1.3 K 
4  | static_variable_selection          | VariableSelectionNetwork        | 1.8 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 55.4 K
6  | decoder_variable_selection         | VariableSelectionNetwork        | 54.8 K
7  | static_context_variable_selection  | GatedResidualNetwork            | 1.1 K 
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 1.1 K 
9  | static_context_initial_cell_lstm   | GatedResidualNetwork            | 1.1 

Sanity Checking DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  9.68it/s]

/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████| 7/7 [00:01<00:00,  4.51it/s, loss=0.518, v_num=33, train_loss_step=0.445, val_loss=2.000]

...

Epoch 11: 100%|██████████| 7/7 [00:01<00:00,  4.62it/s, loss=0.137, v_num=33, train_loss_step=0.101, val_loss=0.060, train_loss_epoch=0.134] 

Metric val_loss improved by 0.002 >= min_delta = 0.0. New best score: 0.060


Epoch 21: 100%|██████████| 7/7 [00:01<00:00,  4.31it/s, loss=0.0785, v_num=33, train_loss_step=0.0758, val_loss=0.0645, train_loss_epoch=0.0796]

Monitored metric val_loss did not improve in the last 10 records. Best score: 0.060. Signaling Trainer to stop.


Epoch 21: 100%|██████████| 7/7 [00:02<00:00,  3.01it/s, loss=0.0785, v_num=33, train_loss_step=0.0758, val_loss=0.0645, train_loss_epoch=0.075] 


In [112]:
best_model = TemporalFusionTransformer.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_model.predict(val_dataloader)

/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


In [113]:
y_true = actuals.detach().cpu().numpy()
y_pred = predictions.detach().cpu().numpy()

In [114]:
smape = SMAPE()(predictions, actuals).item()
mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
r2 = r2_score(y_true, y_pred)

print(f"\n🔍 Evaluation Metrics for Petrol Price:")
print(f"SMAPE: {smape:.2f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.4f}")


🔍 Evaluation Metrics for Petrol Price:
SMAPE: 0.06
MAE: 26.04
RMSE: 26.04
R² Score: nan


/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/sklearn/metrics/_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [119]:
residuals = actuals - predictions  # shape: (num_batches, prediction_length)
residuals_np = residuals.detach().cpu().numpy().flatten()

# Extracting dates from x object used in prediction
# Each sample in x has 'decoder_time_idx' which maps to actual prediction dates
decoded_time = x["decoder_time_idx"].detach().cpu().numpy().flatten()

# Match with dates from the original dataframe
date_lookup = df.set_index("time_idx")["Date"]
dates = pd.to_datetime([date_lookup[int(t)] for t in decoded_time])

# Now build residual DataFrame
res_df = pd.DataFrame({
    "date": dates,
    "residuals": residuals_np
})

In [120]:
res_df.to_csv("tft_petrol_residuals.csv", index=False)

### Training TFT for diesel price prediction

In [131]:
agri_index_df = pd.read_csv("/Users/yehana2002/Projects/DSGP/datasets/raw/diesel_other_factros.csv")
agri_index_df = agri_index_df[["Date", "sl_agri_production_index"]]
agri_index_df["Date"] = pd.to_datetime(agri_index_df["Date"])
agri_index_df["Year"] = agri_index_df["Date"].dt.year
agri_index_df = agri_index_df[["Year", "sl_agri_production_index"]]

In [132]:
final_df = pd.read_csv("/Users/yehana2002/Projects/DSGP/datasets/processed/final_merged_dataset_ready.csv")
final_df["Date"] = pd.to_datetime(final_df["Date"])
final_df["Year"] = final_df["Date"].dt.year

In [133]:
final_df = final_df.merge(agri_index_df, on="Year", how="left")

In [134]:
final_df["sl_agri_production_index"].fillna(method="ffill", inplace=True)

In [136]:
final_df[["Date", "Diesel_Price", "Year", "sl_agri_production_index"]].head()

,Date,Diesel_Price,Year,sl_agri_production_index
0,1992-08-21,14.5,1992,52.99
1,1992-12-21,14.5,1992,52.99
2,1993-06-07,14.5,1993,61.51
3,1993-06-11,14.5,1993,61.51
4,1993-07-12,14.6,1993,61.51


In [137]:
df = df.sort_values("Date").reset_index(drop=True)
df.rename(columns={"Diesel_Price": "diesel_price"}, inplace=True)
df["series_id"] = "sri_lanka"
df["time_idx"] = np.arange(len(df))

In [138]:
max_encoder_length = 36
max_prediction_length = 12
training_cutoff = df["time_idx"].max() - max_prediction_length

time_varying_known_reals = [
    "time_idx", "Date"
] + [col for col in df.columns if col not in ["diesel_price", "petrol_price", "series_id", "time_idx"] and df[col].dtype in [np.float64, np.int64]]

In [139]:
training = TimeSeriesDataSet(
    df[df.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="diesel_price",
    group_ids=["series_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["series_id"],
    time_varying_known_reals=time_varying_known_reals,
    time_varying_unknown_reals=["diesel_price"],
    target_normalizer=GroupNormalizer(groups=["series_id"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)

#### Optuna testing for diesel model

In [140]:
def objective(trial):
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    hidden_size = trial.suggest_int("hidden_size", 8, 64)
    attention_head_size = trial.suggest_int("attention_head_size", 1, 4)
    dropout = trial.suggest_float("dropout", 0.0, 0.3)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)

    train_loader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_loader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    model = TemporalFusionTransformer.from_dataset(
        training,
        hidden_size=hidden_size,
        attention_head_size=attention_head_size,
        dropout=dropout,
        learning_rate=learning_rate,
        loss=SMAPE(),
        log_interval=0,
        reduce_on_plateau_patience=4,
    )

    trainer = Trainer(
        max_epochs=20,
        accelerator="cpu",
        logger=False,
        enable_checkpointing=False,
        callbacks=[EarlyStopping(monitor="val_loss", patience=5)],
    )

    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

    val_actuals = torch.cat([y[0] for x, y in iter(val_loader)])
    val_predictions = model.predict(val_loader)

    smape_score = SMAPE()(val_predictions, val_actuals).item()
    return smape_score

In [141]:
study = optuna.create_study(direction="minimize", study_name="TFT_Diesel_Tuning")
study.optimize(objective, n_trials=30)

# Print results
print("✅ Best SMAPE:", study.best_value)
print("🎯 Best Params:")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")

[I 2025-04-05 16:56:04,565] A new study created in memory with name: TFT_Diesel_Tuning
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS availabl

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 19: 100%|██████████| 2/2 [00:00<00:00, 11.91it/s, loss=0.451, train_loss_step=0.406, val_loss=1.100, train_loss_epoch=0.406]

...

Epoch 19: 100%|██████████| 7/7 [00:00<00:00,  7.30it/s, loss=0.327, train_loss_step=0.228, val_loss=0.101, train_loss_epoch=0.299]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 7/7 [00:00<00:00,  7.29it/s, loss=0.327, train_loss_step=0.228, val_loss=0.101, train_loss_epoch=0.299]

[I 2025-04-05 17:01:53,095] Trial 29 finished with value: 0.1008903905749321 and parameters: {'batch_size': 8, 'hidden_size': 41, 'attention_head_size': 4, 'dropout': 0.2545447585067546, 'learning_rate': 0.0003433546261363879}. Best is trial 4 with value: 0.08354290574789047.



✅ Best SMAPE: 0.08354290574789047
🎯 Best Params:
  batch_size: 16
  hidden_size: 63
  attention_head_size: 3
  dropout: 5.71315030161168e-05
  learning_rate: 0.0013366258010220143


In [153]:
train_loader = training.to_dataloader(train=True, batch_size=16, num_workers=0)
val_loader = validation.to_dataloader(train=False, batch_size=16, num_workers=0)

tft_diesel = TemporalFusionTransformer.from_dataset(
    training,
    hidden_size=63,
    attention_head_size=3,
    dropout=0.01,
    learning_rate=0.0013366258010220143,
    loss=SMAPE(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)
early_stop = EarlyStopping(monitor="val_loss", patience=10, verbose=True, mode="min")
lr_logger = LearningRateMonitor()

/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


In [154]:
trainer = Trainer(
    max_epochs=50,
    accelerator="cpu",
    callbacks=[early_stop, lr_logger],
    gradient_clip_val=0.1,
)

trainer.fit(tft_diesel, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | SMAPE                           | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 1     
3  | prescalers                         | ModuleDict                      | 1.3 K 
4  | static_variable_selection          | VariableSelectionNetwork        | 5.2 K 
5  | encoder_variable_se

Sanity Checking DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  9.74it/s]

/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/anaconda3/envs/fuel_env/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████| 4/4 [00:01<00:00,  2.74it/s, loss=0.482, v_num=41, train_loss_step=0.473, val_loss=1.410]

...

Epoch 32: 100%|██████████| 4/4 [00:01<00:00,  2.94it/s, loss=0.14, v_num=41, train_loss_step=0.139, val_loss=0.103, train_loss_epoch=0.135] 

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.103


Epoch 42: 100%|██████████| 4/4 [00:01<00:00,  2.93it/s, loss=0.105, v_num=41, train_loss_step=0.104, val_loss=0.133, train_loss_epoch=0.0962] 

Monitored metric val_loss did not improve in the last 10 records. Best score: 0.103. Signaling Trainer to stop.


Epoch 42: 100%|██████████| 4/4 [00:02<00:00,  1.97it/s, loss=0.105, v_num=41, train_loss_step=0.104, val_loss=0.133, train_loss_epoch=0.102] 


In [155]:
actuals = torch.cat([y[0] for x, y in iter(val_loader)])
predictions = tft_diesel.predict(val_loader)

y_true = actuals.detach().cpu().numpy()
y_pred = predictions.detach().cpu().numpy()

print(f"\n🔍 Final Evaluation Metrics for Diesel (Optuna-Tuned):")
print(f"SMAPE: {SMAPE()(predictions, actuals).item():.2f}")
print(f"MAE: {mean_absolute_error(y_true, y_pred):.2f}")
print(f"RMSE: {mean_squared_error(y_true, y_pred, squared=False):.2f}")


🔍 Final Evaluation Metrics for Diesel (Optuna-Tuned):
SMAPE: 0.13
MAE: 50.57
RMSE: 50.57


In [156]:
raw_predictions, x = tft_diesel.predict(val_loader, mode="raw", return_x=True)

# Computing residuals
residuals = actuals - predictions
residuals_np = residuals.detach().cpu().numpy().flatten()

# Mapping decoder time_idx to real dates
decoded_time = x["decoder_time_idx"].detach().cpu().numpy().flatten()
date_lookup = df.set_index("time_idx")["Date"]
dates = pd.to_datetime([date_lookup[int(t)] for t in decoded_time])

# Building and saving residual DataFrame
res_df = pd.DataFrame({
    "date": dates,
    "residuals": residuals_np
})

res_df.to_csv("tft_diesel_residuals.csv", index=False)
print("Diesel residuals saved to 'tft_diesel_residuals.csv'")

Diesel residuals saved to 'tft_diesel_residuals.csv'
